# Build Dataset
Format the downloaded articles as a huggingface dataset and push to hub.

In [1]:
import os
from datasets import load_dataset
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
folder = "articles"
files = os.listdir(folder)
files.sort()
files[0:5]

['12-days-of-college-ultimate-2024-things-were-excited-about.txt',
 '12-days-of-college-ultimate-2024-universe-lines.txt',
 '12-days-of-college-ultimate-5-team-strategies.txt',
 '12-days-of-college-ultimate-top-1st-year-players.txt',
 '2013-tidewater-tuneup-tournament-preview.txt']

In [4]:
dataset_name = "ultimate-articles"
dataset = load_dataset("text", 
                       name = dataset_name,
                       data_dir = folder, 
                       data_files = files,
                       num_proc = 2, 
                       sample_by = "paragraph", # or "paragraph" or "document"
)
dataset

Resolving data files:   0%|          | 0/568 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 773
    })
})

In [5]:
n=50
dataset["train"][n]

{'text': "\xa0\nUltiworld's\xa0College Power Rankings, presented by the National Ultimate Training Camp!\nWe are excited to present the 2024 preseason College Power Rankings! As we head into a new season, we've looked at past performance, roster changes, preseason results, and more to figure out how we think the team could stack up this season. Take a look at our first rankings set of the spring college season and read up about each Top 25 team.\nWe will be counting down our rankings in three parts:\nThe #25 to #16\nThe #15 to #6\nThe Top 5\n16. Georgia Athena\nNo beating around the bush here. Georgia returns both Quincy Booth and Fiona Cashin, who put up a combined 20G/43A/24B statline in Cincinnati last year. With a year of college, club (Ozone), and PUL (Atlanta Soul) together, they are ready to power Georgia to another Southeast regional title and the bracket of Nationals. And word on the street is the depth around them has increased considerably.\n– Grace Conerly\n2023 Finish: #23

In [6]:
# add length column
def compute_text_length(example):
    return {"length": len(example["text"].split())}
dataset = dataset.map(compute_text_length)

# filter out short examples
min_length = 8
dataset = dataset.filter(lambda x: x["length"] >= min_length)
dataset

Map:   0%|          | 0/773 [00:00<?, ? examples/s]

Filter:   0%|          | 0/773 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'length'],
        num_rows: 773
    })
})

In [11]:
dataset.push_to_hub(
    repo_id = f'{os.environ["HF_USERNAME"]}/{dataset_name}',
    private = True,
    token = os.environ["HF_TOKEN"]
)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/405 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Roverto/ultimate-articles/commit/eb0ab0ae356dde199d90672504c501ebf35087d1', commit_message='Upload dataset', commit_description='', oid='eb0ab0ae356dde199d90672504c501ebf35087d1', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
n = len(dataset["train"]["text"])
total_length = 0
for item in dataset["train"]["text"]:
    total_length+=len(item)
total_length/n

7505.090556274256